In [1]:
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import requests
import json
import datetime
import numpy as np

In [2]:
def send_email(subject, body, recipient_email):
    # 163邮箱的邮件服务器设置
    smtp_server = "smtp.163.com"
    smtp_port = 25
    sender_email = 'Wdp_bot@163.com'  # 请替换为您的163邮箱地址
    sender_password = "***"  # 请替换为您的163邮箱密码或授权码

    # 创建邮件内容
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = recipient_email
    message["Subject"] = subject
    message.attach(MIMEText(body, "plain"))

    # 连接邮件服务器并发送邮件
    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # 开启TLS加密
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, recipient_email, message.as_string())
        print("邮件发送成功！")
    except Exception as e:
        print("邮件发送失败:", str(e))

In [3]:
def get_two_digits(value):
    return "{:.2f}".format(value)

In [4]:
def get_value_from_dict(data, path):
    keys = path.split('.')
    current_data = data
    for key in keys:
        if key in current_data:
            current_data = current_data[key]
        else:
            return None
    return current_data

In [5]:
def data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce):
    # 发送GET请求
    response1 = requests.get(url1)
    response2 = requests.get(url2)
    # 检查响应状态码，如果状态码为200，则请求成功
    if response1.status_code and response2.status_code == 200:
        # 获取响应数据
        dict1 = response1.json()
        dict2 = response2.json()
        #print(dict2)
        # 处理响应数据
        price1 = get_value_from_dict(dict1, path1)
        price2 = get_value_from_dict(dict2, path2)
        price1h = get_value_from_dict(dict2, path1h)
        
        if token_name2 == "BIT_OP":
            price1 = float(price1[:12])
            price2 = float(price2[:12])
            price2 = price2/10000000000
        elif token_name2 == "PEPE_OP":
            price1 = float(price1[:10])
            price2 = float(price2[:10])
        else:
            price1 = float(price1[:6])
            price2 = float(price2[:6])
        
        if isinstance(price1h, str):    
            price1h = float(price1h[:6])
        price_gap = abs((price1 - price2)/price2*100)
        gap_array[nonce] = get_two_digits(price_gap)
        print(token_name1, price1, token_name2, price2, "1h_change", price1h, gap_array[nonce])
        if price_gap > gap:
            subject = f"警报：{token_name2} potential arbitrage！"
            body = f"{current_time}:警报：{token_name2}代币出现套利空间，目前价差为{gap_array[nonce]},{token_name1}价格为{price1},{token_name2}价格为{price2}！"
            print(f"警报：{token_name2}代币出现套利空间，目前价差为{gap_array[nonce]},{token_name1}价格为{price1},{token_name2}价格为{price2}！")
            if mail_index[nonce] == 0:
                send_email(subject, body, recipient_email)
                mail_index[nonce] = 1
        elif abs(price1h) > 10:
            subject = f"警报：{token_name2} 价格波动超过10%"
            body = f"{current_time}:警报：{token_name2}代币价格出现巨大波动，目前1h变化为{price1h}！"
            print(f"警报：{token_name2}代币价格出现巨大波动，目前1h变化为{price1h}！")
            if mail_index[nonce] == 0:
                send_email(subject, body, recipient_email)
                mail_index[nonce] = 2
        else:
            if mail_index[nonce] == 1:
                subject = f"警报解除：{token_name2} 价差消除"
                body = f"{current_time}:警报解除：{token_name2}代币价差消除，目前价差为{gap_array[nonce]},{token_name1}价格为{price1},{token_name2}价格为{price2}！"
                print(f"{current_time}:警报解除：{token_name2}代币价差消除，目前价差为{gap_array[nonce]},{token_name1}价格为{price1},{token_name2}价格为{price2}！")
                mail_index[nonce] = 0
            elif mail_index[nonce] == 2:
                subject = f"警报：{token_name2} 价格波动消除"
                body = f"{current_time}:警报：{token_name2}代币价格波动消除，目前1h变化为{price1h}！"
                print(f"警报：{token_name2}代币价格波动消除，目前1h变化为{price1h}！")
                mail_index[nonce] = 0
            #else:
                #print(f"{token_name1}价差正常")
    else:
        print("请求失败，状态码：", response1.status_code, response2.status_code)

In [6]:
def MET_function():
    token_name1 = "MET_ETH"
    token_name2 = "MET_OP"
    url1 = "https://api.geckoterminal.com/api/v2/networks/eth/pools/0xceb5c29bde4604296135dd7b027a433fd3633516"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0xa8e4fa96327c5a93e159bd781f3ccfd860519d82"
    path1 = 'data.attributes.base_token_price_usd'
    path2 = 'data.attributes.base_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    gap = 15
    current_time = datetime.datetime.now()
    nonce = 1
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [7]:
def T_function():
    token_name1 = "T_ETH"
    token_name2 = "T_OP"
    url1 = "https://api.geckoterminal.com/api/v2/networks/eth/pools/0x752ebeb79963cf0732e9c0fec72a49fd1defaeac"
    url2 = "https://api.dexscreener.com/latest/dex/pairs/optimism/0xc60d585edf334bed161975c2bc91275123a70859"
    path1 = 'data.attributes.base_token_price_usd'
    path2 = 'pair.priceUsd'
    path1h = 'pair.priceChange.h1'
    gap = 20
    current_time = datetime.datetime.now()
    nonce = 2
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [8]:
def BIT_function():
    token_name1 = "BIT_ETH"
    token_name2 = "BIT_OP"
    url1 = "https://api.geckoterminal.com/api/v2/networks/eth/pools/0x2cc846fff0b08fb3bffad71f53a60b4b6e6d6482"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0x602ef41cf4f6a112f8e3dcbb7660287310a9fb51"
    path1 = 'data.attributes.base_token_price_usd'
    path2 = 'data.attributes.base_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    gap = 20
    current_time = datetime.datetime.now()
    nonce = 3
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [9]:
def PEPE_function():
    token_name1 = "PEPE_ETH"
    token_name2 = "PEPE_OP"
    url1 = "https://api.geckoterminal.com/api/v2/networks/eth/pools/0x11950d141ecb863f01007add7d1a342041227b58"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0x83b6f73ca05826b402343bc61d3c180e1597ae50"
    path1 = 'data.attributes.base_token_price_usd'
    path2 = 'data.attributes.base_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    gap = 25
    current_time = datetime.datetime.now()
    nonce = 4
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [10]:
def DHT_function():
    token_name1 = "DHT_ETH"
    token_name2 = "DHT_OP"
    url1 = "https://api.geckoterminal.com/api/v2/networks/eth/pools/0x8cab3e311702acb64e250926d77134fde604bd4d"
    url2 = "https://api.geckoterminal.com/api/v2/networks/optimism/pools/0x7e1357fa56c0ecfe69daf22e08489ae8de56d86c"
    path1 = 'data.attributes.base_token_price_usd'
    path2 = 'data.attributes.quote_token_price_usd'
    path1h = 'data.attributes.price_change_percentage.h1'
    gap = 20
    current_time = datetime.datetime.now()
    nonce = 5
    data_analysis(url1, url2, token_name1, token_name2, path1, path2, path1h, gap, recipient_email, current_time, nonce)

In [11]:
def main():
    try:
        MET_function()
        T_function()
        BIT_function()
        PEPE_function()
        DHT_function()
        max_gap = max(gap_array)
        max_nonce = np.where(gap_array == max_gap)
        max_gap_token = token_array[max_nonce]
        current_time = datetime.datetime.now()
        global index, err_index
        index = index + 1
        err_index = 0
        print(f"rank:{index},now:{current_time},max gap token:{max_gap_token},gap_value:{max_gap}")
        print("\n")
        # 让程序休眠,单位为秒
        time.sleep(0.5 * 60)
        main()
    except Exception as e:
        index = 0
        err_index = err_index + 1
        print(f"err_rank:{err_index},Error: {e}")
        time.sleep(60)
        if err_index < 25:
            main()

In [12]:
recipient_email = "727226168@qq.com"
token_array = np.array(["None","MET","T","BIT","PEPE",'DHT'])
gap_array = np.zeros(20)
index = 0
err_index = 0
mail_index = np.zeros(20)
main()

MET_ETH 1.2809 MET_OP 1.1492 1h_change 0.0 11.46
T_ETH 0.0245 T_OP 0.0222 1h_change 0 10.36
BIT_ETH 0.079695107 BIT_OP 0.075816946973 1h_change 0.0 5.12
PEPE_ETH 1.35e-06 PEPE_OP 1.47e-06 1h_change 0.0 8.16
DHT_ETH 0.1105 DHT_OP 0.1002 1h_change 0.01 10.28
rank:1,now:2023-07-29 12:50:02.643666,max gap token:['MET'],gap_value:11.46


MET_ETH 1.2809 MET_OP 1.1492 1h_change 0.0 11.46
T_ETH 0.0245 T_OP 0.0222 1h_change 0 10.36
BIT_ETH 0.079695107 BIT_OP 0.075816946973 1h_change 0.0 5.12
PEPE_ETH 1.35e-06 PEPE_OP 1.47e-06 1h_change 0.0 8.16
DHT_ETH 0.1105 DHT_OP 0.1002 1h_change 0.01 10.28
rank:2,now:2023-07-29 12:50:40.736923,max gap token:['MET'],gap_value:11.46


